In [ ]:
!pip install pdfplumber pandas numpy

import re
import numpy as np
import pandas as pd
import pdfplumber

PDF_PATH = "/content/albee_et_al_1970.pdf"
PAGES_1INDEXED = [6, 7]

TARGET = ["SiO2", "MgO", "FeO", "MnO", "CaO", "Cr2O3"]
NUM_RE = re.compile(r"^[\(\-]?\d+(?:\.\d+)?[\)]?$")

In [4]:
def canon_oxide(s: str):
    t = str(s).strip().lower().replace(" ", "")
    t = t.replace("s102","sio2").replace("si02","sio2").replace("sioz","sio2")
    t = t.replace("fe0","feo")
    t = t.replace("cr203","cr2o3").replace("cr2o8","cr2o3")
    if t.startswith("sio2"): return "SiO2"
    if t.startswith("mgo"):  return "MgO"
    if t.startswith("feo"):  return "FeO"
    if t.startswith("mno"):  return "MnO"
    if t.startswith("cao"):  return "CaO"
    if t.startswith("cr2o3"):return "Cr2O3"
    return None

def is_number_token(s: str):
    s2 = str(s).replace(",", "").strip()
    return bool(NUM_RE.match(s2))

def to_float(s: str):
    return float(str(s).replace(",", "").strip().strip("()"))

def cluster_1d(vals, tol):
    """Return cluster centers for 1D values."""
    vals = sorted(vals)
    centers = []
    for v in vals:
        if not centers or abs(centers[-1]["c"] - v) > tol:
            centers.append({"c": v, "n": 1})
        else:
            centers[-1]["c"] = (centers[-1]["c"]*centers[-1]["n"] + v)/(centers[-1]["n"]+1)
            centers[-1]["n"] += 1
    return [c["c"] for c in centers]

def group_words_into_rows(words, y_tol=3.0):
    """
    Group words into rows using y-center clustering.
    Returns list of rows; each row is list of word dicts sorted by x0.
    """
    items = []
    for w in words:
        yc = (w["top"] + w["bottom"]) / 2
        items.append((yc, w))
    items.sort(key=lambda t: t[0])

    rows = []
    for yc, w in items:
        placed = False
        for r in rows:
            if abs(r["yc"] - yc) <= y_tol:
                r["words"].append(w)
                # update center
                r["yc"] = (r["yc"]*r["n"] + yc)/(r["n"]+1)
                r["n"] += 1
                placed = True
                break
        if not placed:
            rows.append({"yc": yc, "n": 1, "words": [w]})

    out = []
    for r in rows:
        r["words"].sort(key=lambda w: w["x0"])
        out.append(r["words"])
    return out

def find_header_band(words, max_lines=40):
    """
    Find the top header band likely containing mineral names.
    We look for a row containing 'olivine' and at least one other capitalized mineral-ish word.
    """
    rows = group_words_into_rows(words, y_tol=4.0)
    for row in rows[:max_lines]:
        texts = [w["text"] for w in row]
        low = [t.lower() for t in texts]
        if any("olivine" in t for t in low):
            return row
    return None

def olivine_x_bounds_from_header_row(header_row):
    """
    Use the header row containing mineral names.
    Bound olivine block as:
      left = olivine x0 - small pad
      right = next header token x0 - small pad  (prevents leaking into next mineral block)
    If no next header, right = olivine x1 + conservative pad.
    """
    # sort by x
    hdr = sorted(header_row, key=lambda w: w["x0"])
    # find olivine token(s)
    olv_idx = [i for i,w in enumerate(hdr) if "olivine" in w["text"].lower()]
    if not olv_idx:
        return None

    i0 = olv_idx[0]
    left = hdr[i0]["x0"] - 5

    # next "header-ish" token to the right (skip tiny symbols/numbers)
    right = None
    for j in range(i0+1, len(hdr)):
        t = hdr[j]["text"].strip()
        if not t:
            continue
        # skip numeric column labels if any
        if is_number_token(t):
            continue
        # skip very short punctuation
        if len(t) <= 1:
            continue
        right = hdr[j]["x0"] - 5
        break

    if right is None:
        right = hdr[i0]["x1"] + 140  # conservative if no neighbor header
    return left, right, hdr[i0]["top"], hdr[i0]["bottom"]

def extract_olivine_only_page(pl_page, debug=False):
    words = pl_page.extract_words(keep_blank_chars=False, use_text_flow=True)
    header_row = find_header_band(words)
    if header_row is None:
        if debug: print("No header row with 'Olivine' found.")
        return pd.DataFrame(columns=["page","olivine_col_idx"] + TARGET)

    bounds = olivine_x_bounds_from_header_row(header_row)
    if bounds is None:
        if debug: print("No olivine bounds.")
        return pd.DataFrame(columns=["page","olivine_col_idx"] + TARGET)

    xL, xR, yH_top, yH_bot = bounds
    # data region starts below header
    y_data_top = yH_bot + 5
    # take most of the page downward (safe); stop before footer if needed
    y_data_bot = pl_page.height - 30

    if debug:
        print("page", pl_page.page_number, "xL,xR:", (xL,xR), "y_data:", (y_data_top,y_data_bot))

    # crop words to the olivine data block
    block = [w for w in words if (xL <= w["x0"] <= xR) and (y_data_top <= w["top"] <= y_data_bot)]
    # also need oxide labels on left of block, so collect label words from full page but within y region
    left_band = [w for w in words if (w["x0"] < xL) and (y_data_top <= w["top"] <= y_data_bot)]

    # group rows for robust row alignment
    rows_block = group_words_into_rows(block, y_tol=3.0)
    rows_left  = group_words_into_rows(left_band, y_tol=3.0)

    # build mapping from row yc -> oxide label (from left_band rows)
    # use closest row match in y between left labels and block rows
    left_labels = []
    for r in rows_left:
        if not r:
            continue
        # take left-most word as potential label
        w0 = r[0]
        ox = canon_oxide(w0["text"])
        if ox:
            yc = (w0["top"] + w0["bottom"]) / 2
            left_labels.append((yc, ox))

    if debug:
        print("oxide labels found:", sorted(set(ox for _,ox in left_labels)))

    # helper: find closest label for a block row y
    def closest_label(yc):
        if not left_labels:
            return None
        return min(left_labels, key=lambda t: abs(t[0]-yc))[1] if min(abs(t[0]-yc) for t in left_labels) < 10 else None

    # find the SiO2 row inside olivine block to infer column centers
    sio2_row_words = None
    sio2_row_yc = None
    for r in rows_block:
        if not r: continue
        yc = (r[0]["top"] + r[0]["bottom"]) / 2
        ox = closest_label(yc)
        if ox == "SiO2":
            sio2_row_words = r
            sio2_row_yc = yc
            break

    if sio2_row_words is None:
        if debug: print("No SiO2 row aligned; cannot infer columns.")
        return pd.DataFrame(columns=["page","olivine_col_idx"] + TARGET)

    # infer analysis column x-centers from numeric tokens in SiO2 row (within olivine block)
    sio2_nums = [w for w in sio2_row_words if is_number_token(w["text"])]
    col_xs = cluster_1d([ (w["x0"]+w["x1"])/2 for w in sio2_nums ], tol=18.0)
    if debug:
        print("inferred olivine analysis columns:", len(col_xs), col_xs)

    if not col_xs:
        return pd.DataFrame(columns=["page","olivine_col_idx"] + TARGET)

    # build oxide->row_words mapping for rows that align to oxide labels
    oxide_to_row = {}
    for r in rows_block:
        if not r: continue
        yc = (r[0]["top"] + r[0]["bottom"]) / 2
        ox = closest_label(yc)
        if ox in TARGET and ox not in oxide_to_row:
            oxide_to_row[ox] = r

    # extract values per column center per oxide
    records = []
    for j, cx in enumerate(col_xs, start=1):
        rec = {"page": pl_page.page_number, "olivine_col_idx": j}
        for ox in TARGET:
            r = oxide_to_row.get(ox)
            if not r:
                rec[ox] = np.nan
                continue
            nums = [w for w in r if is_number_token(w["text"])]
            # pick closest numeric token in x to column center, but only within this row
            best, best_dx = None, 1e9
            for w in nums:
                xw = (w["x0"]+w["x1"])/2
                dx = abs(xw - cx)
                if dx < best_dx:
                    best_dx, best = dx, w
            if best is None or best_dx > 25:
                rec[ox] = np.nan
            else:
                rec[ox] = to_float(best["text"])
        # keep columns that have enough oxides
        if sum(pd.notna(rec[k]) for k in TARGET) >= 3:
            records.append(rec)

    return pd.DataFrame(records, columns=["page","olivine_col_idx"] + TARGET)


In [ ]:
dfs = []
with pdfplumber.open(PDF_PATH) as pdf:
    for p1 in PAGES_1INDEXED:
        dfs.append(extract_olivine_only_page(pdf.pages[p1-1], debug=True))

df_olivine = pd.concat(dfs, ignore_index=True)
display(df_olivine)

df_olivine.to_csv("albee_1970_pages6_7_olivine_only.csv", index=False)
print("Wrote albee_1970_pages6_7_olivine_only.csv")